In [1]:
import netket as nk
import numpy as np
import json
from math import pi

L = 6

In [2]:
# Build square lattice with nearest and next-nearest neighbor edges
lattice = nk.graph.Square(L, max_neighbor_order=2)
hi = nk.hilbert.Spin(s=1 / 2, total_sz=0, N=lattice.n_nodes)
# Heisenberg with coupling J=1.0 for nearest neighbors
# and J=0.5 for next-nearest neighbors
H = nk.operator.Heisenberg(hilbert=hi, graph=lattice, J=[1.0, 0.5])

In [4]:
# Find an approximate ground state
machine = nk.models.GCNN(
    symmetries=lattice,
    parity=1,
    layers=2,
    features=4,
    param_dtype=complex,
)

In [5]:
sampler = nk.sampler.MetropolisExchange(
    hilbert=hi,
    n_chains=1024,
    graph=lattice,
    d_max=2,
)

In [6]:
opt = nk.optimizer.Sgd(learning_rate=0.02)
sr = nk.optimizer.SR(diag_shift=0.01)

In [7]:
vstate = nk.vqs.MCState(
    sampler=sampler,
    model=machine,
    n_samples=1024,
    n_discard_per_chain=0,
    chunk_size=4096,
)

In [8]:
vstate.parameters

{'dense_symm': {'bias': Array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex128),
  'kernel': Array([[[ 0.11131257-0.00678441j, -0.05249117-0.25026485j,
           -0.11513723-0.04899424j,  0.11802233-0.03164709j,
            0.01256458-0.05333783j, -0.17494419-0.22249937j,
           -0.020269  +0.02271964j,  0.1998194 -0.05546632j,
            0.06962062-0.29999884j, -0.11079248-0.00480508j,
            0.00244562+0.00640061j, -0.02137667+0.19967826j,
           -0.06634401-0.10495728j, -0.15724728+0.06933416j,
           -0.10019455-0.03537143j, -0.14762175+0.17029497j,
           -0.03827781-0.01588156j,  0.03057774-0.14298404j,
           -0.0265991 -0.05131403j,  0.15948875+0.04747592j,
           -0.06701911+0.00204035j,  0.07693456-0.08514801j,
            0.05820326+0.02221877j, -0.13642338+0.04256458j,
            0.21572813-0.13533846j,  0.03044077+0.10540994j,
            0.21986783-0.11531138j, -0.08508112-0.26136385j,
           -0.22521486-0.13883607j,  0.13727978-0.015

In [9]:
gs = nk.driver.VMC(H, opt, variational_state=vstate, preconditioner=sr)

In [14]:
gs

Vmc(
  step_count = 0,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0, N=36), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 432), n_chains = 1024, n_sweeps = 36, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1024))

In [ ]:
gs.run(n_iter=200, out="ground_state")

In [ ]:
# Find an approximate ground state
machine = nk.models.GCNN(
    symmetries=lattice,
    parity=1,
    layers=4,
    features=4,
    param_dtype=complex,
)
sampler = nk.sampler.MetropolisExchange(
    hilbert=hi,
    n_chains=1024,
    graph=lattice,
    d_max=2,
)
opt = nk.optimizer.Sgd(learning_rate=0.02)
sr = nk.optimizer.SR(diag_shift=0.01)
vstate = nk.vqs.MCState(
    sampler=sampler,
    model=machine,
    n_samples=1024,
    n_discard_per_chain=0,
    chunk_size=4096,
)
gs = nk.driver.VMC(H, opt, variational_state=vstate, preconditioner=sr)
gs.run(n_iter=200, out="ground_state")

In [ ]:
data = json.load(open("ground_state.log"))
print(np.mean(data["Energy"]["Mean"]["real"][-20:]) / 400)
print(np.std(data["Energy"]["Mean"]["real"][-20:]) / 400)